# <center><a href='https://notebook.basthon.fr/?from=https://raw.githubusercontent.com/lchalmain/mp2i-info/main/tp1-6_types_construits_corrige.ipynb?kernel=ocaml'>TP 1.6 : Types construits<img src=https://framagit.org/uploads/-/system/project/avatar/55763/basthon_shadow.png width=100></a></center>

<center><h1 style = "color:red">Corrigé</h1></center>

 **<span style = "color:orange">Important :</span>** lorsque vous définissez une fonction, pensez à la tester sur un jeu de valeurs !

# Types énumérés

**<p style = "color:purple">Exercice :</p>**
On souhaite simuler l'expérience aléatoire consistant à selectionner successivement et avec remise 10 boules dans une urne contenant :
* 5 boules rouges
* 15 boules bleues
* 20 boules noires numérotées de 1 à 20
1. Déclarer un type `boule` où chacun des constructeurs correspondra aux différentes boules disponibles dans l'urne.
1. À l'aide de la fonction `Random.int : int -> int`, telle que `Random.int n` renvoie un entier aléatoire entre `0` et `n-1`, écrire une fonction `tirage : unit -> boule liste` qui simule l'expérience décrite ci-dessus.
1. Ecrire une fonction points qui détermine le nombre de points obtenus lors d'un tirage sachant que :
    * chaque boule rouge fait perdre 20 points
    * chaque boule bleue fait perdre 2 points
    * chaque boule noire rapporte autant de points qu'indiqué sur la boule
1. Sachant qu'un point rapporte 0,10 € quelle doit être la mise pour que ce jeu soit équitable ? Vérifier ce résultat à l'aide d'une fonction simulant un nombre important de tirages.
<hr>

In [4]:
type boule = Rouge | Bleue | Noire of int

type boule = Rouge | Bleue | Noire of int

In [5]:
let tirage () = 
    let rec tire n = match n with  (* tire n simule un tirage de n boules *)
        | 0 -> []
        | _ -> let b = Random.int 40 in
                    (if b < 5 then Rouge else
                    if b < 20 then Bleue
                    else Noire (b - 19))::(tire (n-1))
    in tire 10
;;

val tirage : unit -> boule list = <fun>

In [7]:
let t = tirage ()

val t : boule list =
  [Bleue; Bleue; Noire 10; Noire 19; Bleue; Bleue; Rouge; Bleue; Noire 9;
   Noire 10]

In [8]:
let rec points tirage = match tirage with
    | [] -> 0
    | Rouge::q -> points q - 20
    | Bleue::q -> points q - 2
    | (Noire n)::q -> points q + n

val points : boule list -> int = <fun>

In [9]:
points t

- : int = 18

In [12]:
let simu nb = 
    let rec tirages n = match n with  (* tirages n renvoie le gain total en euros lors de n tirages simulés *)
        | 0 -> 0.
        | _ -> let p = points (tirage ()) in 
                    ( (float_of_int p) /. 10. ) +. tirages (n-1)  (* chaque point rapporte 0,10 € *)
    in (tirages nb) /. (float_of_int nb)

val simu : int -> float = <fun>

In [75]:
simu 8000

- : float = 1.97556249999999634

In [76]:
simu 10000

Stack overflow during evaluation (looping recursion?).

**<span style="color:red"> Remarque :</span>** la pile d'appels est bien plus limitée lorsqu'on utilise Basthon que lorsqu'on utilise l'interpréteur Ocaml directement sur sa machine.

# Types enregistrement

**<p style = "color:purple">Exercice :</p>** 
1. Déclarer un type `algebrique` correspondant à la description d'un nombre complexe par sa partie réelle et sa partie imaginaire (flottantes).
1. Définir les opérateurs infixes `++`, `--`, `&*` et `&/` qui permetent d'effectuer respectivement la somme, la soustraction, la multiplication et la division de deux nombres complexes. La déclaration se fait comme ceci :
```ocaml
let (++) z1 z2 = ...
```
et l'opérateur s'utilise ensuite comme ceci :
```ocaml
z1 ++ z2;;
```
3. Ecrire une fonction `modulus` qui renvoie le module d'un nombre complexe donné sous forme algébrique.
3. Déclarer un type `exponentielle` correspondant à l'écriture d'un nombre complexe sous la forme $re^{i\theta}$.
3. Ecrire une fonction `alg_of_exp` qui renvoie un nombre complexe, donné sous forme exponentielle, sous forme algébrique.
<hr>

In [32]:
(* 1 *)
type algebrique = { re : float; im : float}

type algebrique = { re : float; im : float; }

In [33]:
let za = { re = 3.; im = -2.};;
let zb = {re = 4.; im = 5.}

val za : algebrique = {re = 3.; im = -2.}
val zb : algebrique = {re = 4.; im = 5.}

In [34]:
(* 2 *)
let (++) z1 z2 = {
    re = z1.re +. z2.re;
    im = z1.im +. z2.im
};;

assert (za ++ zb = {re = 7.; im = 3.});;

val ( ++ ) : algebrique -> algebrique -> algebrique = <fun>
- : unit = ()

In [35]:
let (--) z1 z2 = {
    re = z1.re -. z2.re;
    im = z1.im -. z2.im
};;

assert (za -- zb= {re = -1.; im = -7.});;

val ( -- ) : algebrique -> algebrique -> algebrique = <fun>
- : unit = ()

In [36]:
let (&*) z1 z2 = {
    re = z1.re *. z2.re -. z1.im *. z2.im;
    im = z1.im *. z2.re +. z1.re *. z2.im
};;

assert (za &* zb= {re = 22.; im = 7.});;

val ( &* ) : algebrique -> algebrique -> algebrique = <fun>
- : unit = ()

In [37]:
let (&/) z1 z2 = {
    re = (z1.re *. z2.re +. z1.im *. z2.im) /. (z2.re ** 2. +. z2.im ** 2.);
    im = (z1.im *. z2.re -. z1.re *. z2.im) /. (z2.re ** 2. +. z2.im ** 2.)
};;

assert ( -0.0001 < (za &/ zb).re -. 2. /. 41. && (za &/ zb).re -. 2. /. 41. < 0.0001);
assert ( -0.0001 < (za &/ zb).im +. 23. /. 41. && (za &/ zb).im +. 23. /. 41. < 0.0001);;

val ( &/ ) : algebrique -> algebrique -> algebrique = <fun>
- : unit = ()

In [40]:
(* 3 *)
let modulus z = sqrt (z.re ** 2. +. z.im ** 2.);;

assert (modulus { re = 3.; im = -4.} = 5.);;
(* fort heureusement, le test d'égalité entre flottants peu aussi bien se passer ! *)

val modulus : algebrique -> float = <fun>
- : unit = ()

In [41]:
(* 4 *)
type exponentielle = { r : float; theta : float }

type exponentielle = { r : float; theta : float; }

In [53]:
(* 5 *)
let alg_of_exp z = {
    re = z.r *. (cos z.theta);
    im = z.r *. (sin z.theta)
};;

assert (alg_of_exp {r = 5.; theta = 0.} = { re = 5.; im = 0.});;
sqrt 3.;;
alg_of_exp {r = 2.; theta = -. Float.pi /. 6.}    (* cf ci-dessous *)

val alg_of_exp : exponentielle -> algebrique = <fun>
- : unit = ()
- : float = 1.73205080756887719
- : algebrique = {re = 1.73205080756887742; im = -0.999999999999999889}

$$2e^{-\frac{i\pi}6} = 2\left[ \cos \left( - \frac{\pi}6 \right) + i\sin \left( - \frac{\pi}6 \right)\right] = 2\frac{\sqrt3}2 - 2i\times\frac12 = \sqrt3 - i$$

**<p style = "color:purple">Exercice :</p>** 
Dans tout cet exercice, on n'utilisera que des coordonnées entières.
1. Déclarer un type `coord` correspondant à des coordonnées entières en dimensions 3.
1. Ecrire une fonction `orthogonaux : coord -> coord -> bool` qui teste si deux vecteurs de l'espace sont orthogonaux ou non.
1. Ecrire une fonction `vecteur : coord -> coord -> coord` qui renvoie les coordonnées du vecteur d'origine le premier argument et d'extrémité le second argument.
1. Déclarer un type `droite` correspondant aux droites définies par un point et un vecteur directeur.
1. Ecrire une fonction `colinéaires : coord -> coord -> bool` qui teste si deux vecteurs de l'espace sont colinéaires ou non.
1. Ecrire une fonction `appartient : coord -> droite -> bool` telle que `appartient point droite`teste si un point appartient à une droite ou non.
<hr>

In [22]:
type coord = { x : int; y : int; z : int}

type coord = { x : int; y : int; z : int; }

In [23]:
let orthogonaux u v = u.x * v.x + u.y * v.y  + u.z * v.z = 0;;

assert (orthogonaux { x = 1; y = 3; z = 0 } { x = -3; y = 1; z = 4});
assert (not (orthogonaux { x = 1; y = 5; z = 0 } { x = -3; y = 1; z = 4}));;

val orthogonaux : coord -> coord -> bool = <fun>
- : unit = ()

In [24]:
let vecteur a b = {x = b.x - a.x; y = b.y - a.y; z = b.z - a.z};;

assert (vecteur { x = 1; y = 3; z = 0 } { x = -3; y = 1; z = 4} = { x = -4; y = -2; z = 4 });;

val vecteur : coord -> coord -> coord = <fun>
- : unit = ()

In [25]:
type droite = {a : coord; u : coord}

type droite = { a : coord; u : coord; }

In [26]:
let d = {a = {x=2; y=5; z=0}; u = {x=7; y=4; z=2}}

val d : droite = {a = {x = 2; y = 5; z = 0}; u = {x = 7; y = 4; z = 2}}

In [54]:
let colineaires u v =        
    (u.x <> 0 && u.y * v.x = u.x * v.y && u.z * v.x = u.x * v.z)  (* teste si v.y et v.z s'obtiennent par *)
    || (u.x = 0 && v.x = 0 && u.y <> 0 && u.z * v.y = u.y * v.z )           (* un produit en croix *)
    || (u.x = 0 && v.x = 0 && u.y = 0 && v.y = 0)
    || u = { x = 0; y = 0; z = 0 }
;;

assert (colineaires { x = 0; y = 0; z = 0} { x = -9; y = 3; z = 12});
assert (colineaires { x = -3; y = 1; z = 4} { x = 0; y = 0; z = 0});
assert (colineaires { x = 0; y = 0; z = 5} { x = 0; y = 0; z = -2});
assert (colineaires { x = -3; y = 1; z = 4} { x = -9; y = 3; z = 12});
assert (not (colineaires { x = 0; y = 1; z = 4} { x = 5; y = 3; z = 12}));;

val colineaires : coord -> coord -> bool = <fun>
- : unit = ()

In [21]:
let appartient point droite = point = droite.a || colineaires (vecteur point droite.a) droite.u;;
(* teste si le vecteur AB est colineaire au vecteur directeur u *)

assert (appartient {x = 9; y = 9 ; z = 2 } d);
assert (not (appartient {x = 9; y = 5 ; z = 2 } d));;

val appartient : coord -> droite -> bool = <fun>
- : unit = ()